In [8]:
from DECAF import ConfigReader

c = ConfigReader.read_config("tests/config/processing/hello_world.ini")
f = ConfigReader.parse_functions(c)
print(f)

['print']


In [7]:
from DECAF import DatafileIO as dio

dio.read_functions_from_ini("tests/config/processing/fuzzy_match_company.ini")

{'functions': ['FuzzyMatch.fuzzy_match_pairwise',
  'FuzzyMatch.score_threshold'],
 'cleaning': ['StringFormat.to_lower',
  'StringFormat.trim_whitespace',
  'StringFormat.strip_non_ascii',
  'StringFormat.strip_punctuation',
  'FuzzyMatch.remove_company_suffixes']}

In [11]:
from DECAF import ComposeFunction as cf

print(cf.eval_functions_list(" hello ", ["DECAF.StringFormat.trim_whitespace"]))
print(cf.eval_functions_list(" ALLCAPS ", ["StringFormat.trim_whitespace", "StringFormat.to_lower"]))

hello
allcaps


In [5]:
from DECAF import DECAF as d

input_file = "tests/data/fuzzy_match_companies.xlsx"
col1="reference_name"
col2="new_name"

d.fuzzy_match_companies(input_file, col1, col2)